In [3]:
from run import Prepro
import numpy as np

In [4]:
train_test_split = 0.2 #how much train test data
predict_at = 8 #steps_out
window_size = 8 #how many timesteps from the past to take ie steps_in

vision_data = np.load('data/vision_data_50_20_60_3years_v2.npy', allow_pickle = True)
y = np.load('data/y_50_20_60_3years_v2.npy', allow_pickle = True)
train_tensors, test_tensors = Prepro.process(vision_data, y, train_test_split, predict_at = 8, window_size =  8)
x_viz_train, x_stat_train, tgt_intensity_cat_train, tgt_intensity_cat_baseline_train, tgt_displacement_train, tgt_intensity_train = train_tensors
x_viz_test, x_stat_test, tgt_intensity_cat_test, tgt_intensity_cat_baseline_test, tgt_displacement_test, tgt_intensity_test = test_tensors

New dataset and corresponding sizes (null elements included):
X_vision torch.Size([3735, 8, 9, 25, 25])
X_stat torch.Size([3735, 8, 10])
target_displacement torch.Size([3735, 8, 2])
target_intensity torch.Size([3735])
target_intensity_cat torch.Size([3735])
target_intensity_cat_baseline torch.Size([3735])
Keeping 3143 samples out of the initial 3735.
Reshaping the displacement target...


In [5]:
x_stat_train[0,1,:]
#'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED', 'STORM_DIR', 'STORM CATEGORY', 
#'STORM_DISPLACEMENT_X', 'STORM_DISPLACEMENT_Y'

tensor([ 6.8000e+00, -1.7200e+02,  5.5000e+01,  9.9400e+02,  2.1470e+03,
         8.0000e+00,  3.2100e+02,  1.0000e+00,  4.1247e-01, -3.0800e-01])

In [21]:
tgt_displacement_train

tensor([[-0.1960,  0.0601],
        [-0.2376,  0.1446],
        [-0.2534,  0.2426],
        ...,
        [ 2.5309, -0.2668],
        [ 2.5147, -0.2978],
        [ 2.4879, -0.2972]])